In [14]:
from autogluon.tabular import TabularDataset, TabularPredictor

import autosklearn.regression
import pandas as pd
import numpy as np
import os
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import argparse
import logging

from sklearn.metrics import accuracy_score, r2_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from IPython.display import Image, display

from datetime import datetime, timedelta

from scipy.stats import entropy

from __future__ import annotations

from pathlib import Path

from autosklearn.regression import AutoSklearnRegressor

In [3]:
random_seed = 42

In [9]:
## Function for loading one of the 10 folds of the property dataset and concatinating the X and y values for train and test respectively.

base_path = '../../data/361092' # Use this when running locally

def load_fold(fold_number, random_seed=random_seed, auto_gluon=False):
    df_X_train = pd.read_parquet(f'{base_path}/{fold_number}/X_train.parquet')
    df_y_train = pd.read_parquet(f'{base_path}/{fold_number}/y_train.parquet')
    df_X_test = pd.read_parquet(f'{base_path}/{fold_number}/X_test.parquet')
    df_y_test = pd.read_parquet(f'{base_path}/{fold_number}/y_test.parquet')

    # concatinating the X and y values for train (and test), but keeping for test
    df_train = pd.concat([df_X_train, df_y_train], axis=1)


    # Convert to AutoGluon's TabularDataset
    if not auto_gluon:
        return df_X_train, df_y_train, df_X_test, df_y_test
    
    else:
        df_test = pd.concat([df_X_test, df_y_test], axis=1)
        train_dataset = TabularDataset(df_train)
        test_dataset = TabularDataset(df_test)

        return train_dataset, test_dataset

# Also instantiate the target column
label_property = 'oz252'

# First we begin with AutoGluon run

# Then we do the AUtoSklearn run

In [12]:
# Make set out of half the folds

full_train_X = None
full_train_y = None
full_test_X = None
full_test_y = None

for fold_number in range(1, 6):
    train_dataset_X, train_dataset_y, test_dataset_X, test_dataset_y = load_fold(fold_number, random_seed=random_seed, auto_gluon=False, )
    if full_train_X is None:
        full_train_X = train_dataset_X
        full_train_y = train_dataset_y
        full_test_X = test_dataset_X
        full_test_y = test_dataset_y
    else:
        # Use pd.concat to combine TabularDatasets
        full_train_X = pd.concat([full_train_X, train_dataset_X])
        full_train_y = pd.concat([full_train_y, train_dataset_y])
        full_test_X = pd.concat([full_test_X, test_dataset_X])
        full_test_y = pd.concat([full_test_y, test_dataset_y])

In [15]:
# Initialize AutoSklearnRegressor
autosklearn_regressor = AutoSklearnRegressor(
    time_left_for_this_task=1000,
    tmp_folder='/tmp/autosklearn_regression_example_tmp',
    
)

In [16]:
# Fit the model on the full training data
autosklearn_regressor.fit(full_train_X, full_train_y)


[ERROR] [2024-07-11 08:30:46,376:Client-AutoML(1):dda875a4-3f5f-11ef-80a5-fa63b2a73ced] (" Dummy prediction failed with run state StatusType.MEMOUT and additional output: {'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'DUMMY'}.",)
[ERROR] [2024-07-11 08:30:46,377:Client-AutoML(1):dda875a4-3f5f-11ef-80a5-fa63b2a73ced] (" Dummy prediction failed with run state StatusType.MEMOUT and additional output: {'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'DUMMY'}.",)
Traceback (most recent call last):
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/autosklearn/automl.py", line 765, in fit
    self._do_dummy_prediction()
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/autosklearn/automl.py", line 489, in _do_dummy_prediction
    raise ValueError(msg)
ValueError: (" Dummy prediction failed with run state StatusType.MEMOUT and additional output: {'error': 'Memout (used more than 3072 MB).', 'co

ValueError: (" Dummy prediction failed with run state StatusType.MEMOUT and additional output: {'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'DUMMY'}.",)